In [1]:
import torch
from torch.utils.data import DataLoader, RandomSampler
from scripts.pums_downloader import datasets

from scripts.pums_sgd import PumsModule, load_pums, problem, evaluate, printf

In [2]:
train_loader = DataLoader(load_pums(datasets[1]), batch_size=10000, shuffle=True)
test_loader = DataLoader(load_pums(datasets[1]), batch_size=10000)
# train_loader.sampler = RandomSampler(replacement=True)

In [3]:
model_ = PumsModule(len(problem['predictors']), 2)
gradients = dict((name, []) for name, _ in model_.named_parameters())

printf(f"acc\t|\tloss")
for batch in train_loader:
    model = PumsModule(len(problem['predictors']), 2)
    optimizer = torch.optim.SGD(model.parameters(), .1)
    loss = model.loss(batch)
    loss.backward()
    # gradients.append(list(model.linear1.parameters()))
    for name, param in model.named_parameters():
        if param.requires_grad:
            grad = param.grad.detach().clone()
            gradients[name].append(grad)
    optimizer.step()
    # print(model.linear1.weight.grad)
    # print(model.linear1.bias.grad)

    optimizer.zero_grad()

    accuracy, loss = evaluate(model, test_loader)
    printf(f"{accuracy.item():.2f}\t|\t{loss.item():.2f}", force=True)

acc	|	loss
0.93	|	135.83
0.93	|	0.58
0.93	|	5.82
0.07	|	20.66


In [7]:
# This should be a dp median
import torch.nn as nn
avg_linear1 = sum(gradients['linear1.weight'])  / len(gradients['linear1.weight'])
avg_linear1_bias = sum(gradients['linear1.bias']) / len(gradients['linear1.bias'])
avg_linear2 = sum(gradients['linear2.weight']) / len(gradients['linear2.weight'])
avg_linear2_bias = sum(gradients['linear2.bias']) / len(gradients['linear2.bias'])

model.linear1.weight = nn.Parameter(avg_linear1)
model.linear1.bias = nn.Parameter(avg_linear1_bias)
model.linear2.weight = nn.Parameter(avg_linear2)
model.linear2.bias = nn.Parameter(avg_linear2_bias)

acc, loss = evaluate(model, test_loader)
acc, loss

UsageError: Line magic function `%%` not found.


In [ ]:
gradients

In [5]:
import numpy as np
import opendp.smartnoise.core as sn

with sn.Analysis():
    grads = [x.mean().item() for x in gradients['linear1.weight']]
    dataset = sn.literal(value=grads, value_public=False)
    typed = sn.cast(dataset, atomic_type={
        bool: 'bool', float: 'float', int: 'int', str: 'str'
    }[float], true_label=True, lower=0, upper=10)
    resized = sn.resize(typed, number_columns=1, lower=0., upper=10.)
    data = sn.to_dataframe(resized, names=['linear1.weight'])
    print(data)
    candidates = [0., 3., 4., 7., 10., 12.]

    median_scores = sn.median(
         data,
         candidates=candidates,
         data_rows=len(gradients))
    # print(median_scores.value)
    # dp_median = sn.exponential_mechanism(median_scores, candidates=candidates, privacy_usage={"epsilon": 1.})
    # print(dp_median.value)

ToDataframe(
  data=Resize(
    data=Cast(
      data=[3.071223020553589, 1.9098789691925049, 1.085517168045044, 0.7798864841461182],
      true_label=True,
      lower=0,
      upper=10,
      atomic_type=float
    ),
    number_columns=1,
    lower=0.0,
    upper=10.0
  ),
  names=['linear1.weight']
)


In [4]:
for batch in train_loader:
    for x in batch:
        print(x.size())
        break

torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([1000, 3])
torch.Size([949, 3])


In [6]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.grad)

linear1.weight tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
linear1.bias tensor([0., 0., 0., 0., 0.])
linear2.weight tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
linear2.bias tensor([0., 0.])


In [7]:
for param in model.parameters():
    print(param.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0., 0., 0., 0.])
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
tensor([0., 0.])


In [8]:
for x,y in model.state_dict().items():
    print(x)
    print(y)

linear1.weight
tensor([[-0.1316, -0.0025,  0.1985],
        [-1.0574, -2.3805, -0.5121],
        [ 0.2873, -0.5425, -0.2715],
        [-0.2792, -0.0577,  0.0040],
        [-0.1770,  0.2283, -0.0225]])
linear1.bias
tensor([ 0.0158, -0.2395,  0.4055, -0.3556,  0.1591])
linear2.weight
tensor([[ 0.1364, -0.3030, -0.1385,  0.1452, -0.1271],
        [-0.0288,  0.2484,  0.0787, -0.2871,  0.0441]])
linear2.bias
tensor([-0.3465,  0.6554])


{'public': False, 'year': 2010, 'record_type': 'person', 'state': 'ct'}